<h1> HHC Raw Data Load Process </h1>

This process is to take the raw data files provided, clean them, and store them into a relational database.

<h4> #1 Import necessary packages </h4>

* ast is used to interpert string data as it's literal type (string to dictionary conversion)
* getpass is used to store and pass DB password
* pandas a dataframe package used in this process for json to rds mapping
* psycopg2 package used to interact with Postgres DBS, the rds used in this project

    
<h4> #2 Setup conection to our postgres database: hhc_db1 </h4>
   
<h4> #3 Load the SSI data </h4>

* optional:  clean the data
* create an empty table 
* insert all records into the SSI table
    
<h4> #4 Load the mexia data </h4>

* optional:  clean the data
* create an empty table 
* insert all records into the mexia table

<h4> #5 Close connections to Postgres </h4>
    

<h2> Step 1 </h2>

In [1]:
from getpass import getpass
import ast
import pandas as pd
import psycopg2

<h2> Step 2 </h2>

In [4]:
pa = getpass('please enter the db admin password:')
conn = psycopg2.connect("dbname=hhc_db1 user=postgres password=%s" %pa)
cur = conn.cursor()

please enter the db admin password:········


<h2> Step 3 - SSI Data Load </h2>

<h6> create and load the socialsignin_profiles  </h6>

In [ ]:
#profilesCleaned.txt is derived form the master file, the only change is that
#ssi_id in the first column has been cleansed so the json data can be extracted

oldFileName = 'ss_profile/v2/profilesCleaned.txt'
newFileName = 'ss_profile/v2/2017_01_11_socialsignin_profilesFormatted.csv'
skippedRows = 0
listedDicts = []

#pull the data from txt file.
with open(oldFileName, 'r') as f:
    rawString = f.read()

#correct for true/false formatting 
rawStringClean1 = rawString.replace('true', '"true"')
rawStringClean2 = rawStringClean1.replace('false', '"false"')

#split raw data string into a list of strings.
listedStrings= rawStringClean2.split("\r")

#convert the list of strings into a list of dictionaries
for string in listedStrings:
    try:
        listedDicts.append(ast.literal_eval(string))
    except:
        #due to " and ' some data is no longer in a reasonable format.
        pass
        skippedRows += 1

#print records skipped
print str(skippedRows) + ' rows skipped'

#format data using Pandas
ss_profile = pd.DataFrame(listedDicts)

#push data to csv in order to prepare for RDS load
ss_profile.to_csv(newFileName,sep='|',index=False, header=False)


#build new empty table on RDS
cur.execute('''CREATE TABLE ss_profiles 
                   (keyId serial Primary Key,
                    created varchar,
                    default_login_method varchar,
                    email varchar,
                    email_opt_in varchar,
                    fb_birthday varchar,
                    fb_email varchar,
                    fb_fav_athletes varchar,
                    fb_fav_team varchar,
                    fb_frst_nm varchar,
                    fb_gender varchar,
                    fb_hometown varchar,
                    fb_id varchar,
                    fb_insprd_ppl varchar,
                    fb_lngs varchar,
                    fb_lst_nm varchar,
                    fb_locale varchar,
                    fb_loc varchar,
                    fb_md_nm varchar,
                    fb_nm varchar,
                    fb_sprts varchar,
                    fb_tz varchar,
                    fb_udt_tm varchar,
                    fb_usr_nm varchar,
                    fb_vrfd varchar,
                    fb_id_2 varchar,
                    first_ap varchar,
                    frst_nm varchar,
                    gender varchar,
                    last_ap varchar,
                    lst_nm varchar,
                    lst_sn varchar,
                    loc_nm varchar,
                    name varchar,
                    phone varchar,
                    ssi_id varchar,
                    vst_count varchar
                    );''')


#insert data into table
f = open(newFileName)
cur.copy_from(f, 'ss_profiles', sep='|', columns=('created',
                                                'default_login_method',
                                                'email',
                                                'email_opt_in',
                                                'fb_birthday',
                                                'fb_email',
                                                'fb_fav_athletes',
                                                'fb_fav_team',
                                                'fb_frst_nm',
                                                'fb_gender',
                                                'fb_hometown',
                                                'fb_id',
                                                'fb_insprd_ppl',
                                                'fb_lngs',
                                                'fb_lst_nm',
                                                'fb_locale',
                                                'fb_loc',
                                                'fb_md_nm',
                                                'fb_nm',
                                                'fb_sprts',
                                                'fb_tz',
                                                'fb_udt_tm',
                                                'fb_usr_nm',
                                                'fb_vrfd',
                                                'fb_id_2',
                                                'first_ap',
                                                'frst_nm',
                                                'gender',
                                                'last_ap',
                                                'lst_nm',
                                                'lst_sn',
                                                'loc_nm',
                                                'name',
                                                'phone',
                                                'ssi_id',
                                                'vst_count'))
#save changes
conn.commit()


<h6> create and load the socialsignin_mac_references table </h6>

In [ ]:
#build new empty table on RDS
cur.execute('''CREATE TABLE ss_mac_ref 
                   (keyId serial Primary Key,
                    ssi_id varchar,
                    location_id varchar,
                    user_mac varchar,
                    expires_at varchar
                    );''')


#insert data into table
f = open('rawData/2017_01_11_socialsignin_mac_references000.csv')
cur.copy_from(f, 'ss_mac_ref', sep='|', columns=('ssi_id',
                                                 'location_id',
                                                 'user_mac',
                                                 'expires_at'))
#save changes
conn.commit()

<h6> create and load the socialsignin_events table </h6>

In [ ]:
#build new empty table on RDS
cur.execute('''CREATE TABLE ss_events 
                   (keyId serial Primary Key,
                    ssi_id varchar,
                    experience_id varchar,
                    card_id varchar,
                    meta varchar,
                    time_stamp varchar, 
                    channel varchar
                    );''')

#insert data into table
f = open('rawData/2017_01_11_socialsignin_events000.csv')
cur.copy_from(f, 'ss_events', sep='|', columns=('ssi_id',
                                                'experience_id',
                                                'card_id',
                                                'meta',
                                                'time_stamp',
                                                'channel'))
#save changes
conn.commit()

<h6> create and load the socialsignin_interests table </h6>

In [ ]:
#build new empty table on RDS
cur.execute('''CREATE TABLE ss_interests 
                   (keyId serial Primary Key,
                    ssi_id varchar,
                    fb_id varchar,
                    cat varchar,
                    created varchar,
                    name varchar, 
                    id varchar,
                    cache_timestamp varchar
                    );''')

#insert data into table
f = open('rawData/2017_01_11_socialsignin_interests000.csv')
cur.copy_from(f, 'ss_interests', sep='|', columns=('ssi_id',
                                                'fb_id',
                                                'cat',
                                                'created',
                                                'name',
                                                'id',
                                                'cache_timestamp'))
#save changes
conn.commit()

<h6> create and load the socialsignin_interests table </h6>

In [ ]:
#build new empty table on RDS
cur.execute('''CREATE TABLE ss_visits 
                   (keyId serial Primary Key,
                    ssi_id varchar,
                    experience_id varchar,
                    verified_location varchar,
                    visit_date date,
                    visit_timestamp varchar, 
                    total_visits integer
                    );''')

#insert data into table
f = open('rawData/2017_01_11_socialsignin_visits000.csv')
cur.copy_from(f, 'ss_visits', sep='|', columns=('ssi_id',
                                                'experience_id',
                                                'verified_location',
                                                'visit_date',
                                                'visit_timestamp',
                                                'total_visits'))
#save changes
conn.commit()

<h2> Step 4 - Mexia Data Load</h2>

<h6> create and load the mexia visit table </h6>

In [ ]:
#build new empty table on RDS
cur.execute('''CREATE TABLE mexia_ssi_data 
                   (keyId serial Primary Key,
                    mac_address varchar,
                    visit_id varchar,
                    length_of_current_visit_in_sec INTEGER,
                    site_entry_marker varchar,
                    current_zone_id integer,
                    current_zone_name varchar,
                    zone_entry_ts timestamp,
                    zone_exit_ts timestamp,
                    dwell_time_in_sec integer,
                    previous_zone_id integer,
                    previous_zone_name varchar,
                    data_file_date timestamp,
                    created_at timestamp
                    );''')

#insert data into table
f = open('rawData/2017_01_11_unload_mexia_visits.csv')
cur.copy_from(f, 
              'mexia_ssi_data', 
              sep='|', 
              columns=( 'mac_address',
                        'visit_id',
                        'length_of_current_visit_in_sec',
                        'site_entry_marker',
                        'current_zone_id',
                        'current_zone_name',
                        'zone_entry_ts',
                        'zone_exit_ts',
                        'dwell_time_in_sec',
                        'previous_zone_id',
                        'previous_zone_name',
                        'data_file_date',
                        'created_at'
                       ))

#save changes
conn.commit()

<h6> create and load the mexia zone table </h6>

In [5]:


#build new empty table on RDS
cur.execute('''CREATE TABLE mexia_zone_data 
                   (keyId serial Primary Key,
                    zone_id integer,
                    zone_name varchar,
                    x_position varchar,
                    y_position varchar,
                    z_position varchar,
                    zone_group_id integer,
                    zone_type_id integer,
                    area varchar,
                    data_file_date timestamp,
                    created_at timestamp
                    );''')

#insert data into table
f = open('rawData/2017_01_11_unload_mexia_zones.csv')
cur.copy_from(f, 
              'mexia_zone_data', 
              sep='|', 
              columns=( 'zone_id',
                        'zone_name',
                        'x_position',
                        'y_position',
                        'z_position',
                        'zone_group_id',
                        'zone_type_id',
                        'area',
                        'data_file_date',
                        'created_at'
                       ))

#save changes
conn.commit()

<h2> Step 5 </h2>

In [30]:
cur.close()
conn.close()